In [1]:
import pandas as pd
import random
import re
import codecs

In [2]:
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

E:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
E:\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
from pyltp import Segmentor
from pyltp import Postagger
from pyltp import SementicRoleLabeller
from pyltp import NamedEntityRecognizer
from pyltp import Parser

In [8]:
# segmentor = Segmentor()  # 初始化实例
# segmentor.load('../ltp_data/cws.model')  # 加载模型
# postagger = Postagger() # 初始化实例
# postagger.load('../ltp_data/pos.model')  # 加载模型
# parser = Parser() # 初始化实例
# parser.load('../ltp_data/parser.model')  # 加载模型

# # print ("\t".join("%d:%s" % (arc.head, arc.relation) for arc in arcs))
# # parser.release()  # 释放模型
# def segment_pos(sentence):
#     words = segmentor.segment(sentence)  # 分词
#     words_list = list(words)
#     postags = postagger.postag(words)  # 词性标注
#     word_pos=[]
#     arcs = parser.parse(words, postags)  # 句法分析
#     for it in zip(words_list,postags,arcs):
#         word_pos.append((it[0],it[1],it[2].head,it[2].relation))
# #     print ("\t".join("%d:%s" % (arc.head, arc.relation) for arc in arcs))
    
#     # postagger.release()  # 释放模型
#     # segmentor.release()  # 释放模型
#     return word_pos

In [138]:
# segment_pos('收到了，太实惠了，买了一大箱，以后继续购买，送货速度快服务也好')

In [278]:
#train
import json
with open('tmp_train.json') as f:
    col_list=json.load(f)

In [279]:
train=pd.DataFrame()
for index,it in enumerate(col_list):
    train[index]=it

In [281]:
train.columns=['content','theme','sentiment','tokens','postags']
# train.columns=['content','tokens','postags']

In [282]:
orig_seg=[]
for i in range(train.shape[0]):
    token=train.loc[i]['tokens']
    postag=train.loc[i]['postags']
    sent_pos=[]
    for ind in range(len(token)):
        sent_pos.append([(to,po) for to,po in zip(token[ind],postag[ind])])
    orig_seg.append(sent_pos)
train['orig_seg']=orig_seg

In [302]:
wl=[]
for row in train['theme']:
    word=row.split(';')[:-1]
    wlen=[len(w) for w in word if w !='NULL']
    wl.extend(wlen)

In [303]:
from collections import Counter
Counter(wl)

Counter({1: 1091, 2: 17931, 3: 1763, 4: 1216, 5: 12, 6: 4})

In [293]:
#test
with open('tmp_test.json') as f:
    col_list=json.load(f)
test=pd.DataFrame(col_list[0])
for index,it in enumerate(col_list[1:]):
    test[index]=it
test.columns=['content','tokens','postags']

In [294]:
test.shape

(20000, 3)

In [295]:
orig_seg=[]
for i in range(test.shape[0]):
    try:
        token=test.loc[i]['tokens']
        postag=test.loc[i]['postags']
        sent_pos=[]
        for ind in range(len(token)):
            sent_pos.append([(to,po) for to,po in zip(token[ind],postag[ind])])
        orig_seg.append(sent_pos)
    except:
        print(token)
test['orig_seg']=orig_seg

In [296]:
orig_seg[0]

[[('比', 'p'), ('黑人', 'n'), ('碳头', 'n'), ('牙刷', 'n'), ('好', 'a'), ('用', 'v')]]

In [285]:
def get_labels(row):
    word2label=[]
    orig_seg=row['orig_seg']
    try:
        theme=row['theme'].split(';')[:-1]
        sentiment_word=row['sentiment'].split(';')[:-1]
        theme.extend(sentiment_word)
        will_del='\t'.join([it for it in theme if it !='NULL'])
        for sub_sent in orig_seg:
            tmp_lb=[]
            for word in sub_sent:
                if word[0] in will_del:
                    tmp_lb.append((word[0],word[1],'N'))
                else:
                    tmp_lb.append((word[0],word[1],'Y'))
            word2label.append(tmp_lb)
    except:
        for sub_sent in orig_seg:
            tmp_lb=[]
            for word in sub_sent:
                    tmp_lb.append((word[0],word[1],'Y'))
            word2label.append(tmp_lb)
    return word2label

In [286]:
labels=[]
for i in range(train.shape[0]):
    try:
        labels.append(get_labels(train.loc[i]))
    except:
        labels.append([])
train['labels']=labels

In [287]:
labels[0]

[[('收到', 'v', 'Y'), ('了', 'u', 'Y')],
 [('太', 'd', 'Y'), ('实惠', 'a', 'N'), ('了', 'u', 'Y')],
 [('买', 'v', 'Y'),
  ('了', 'u', 'Y'),
  ('一', 'm', 'Y'),
  ('大', 'a', 'Y'),
  ('箱', 'q', 'Y')],
 [('以后', 'nd', 'Y'), ('继续', 'v', 'Y'), ('购买', 'v', 'Y')],
 [('送货', 'v', 'N'),
  ('速度', 'n', 'N'),
  ('快', 'a', 'N'),
  ('服务', 'v', 'N'),
  ('也好', 'u', 'N')]]

In [49]:
with codecs.open('resource/perception.txt',encoding='utf-8') as f:
    percept_words=[line.strip() for line in f]
percept_words=set(percept_words)
def perception(word):
    if word in percept_words:
        return True
    else:
        return False

In [50]:
with codecs.open('resource/polarity_words_1.txt',encoding='utf-8') as f:
    polar_words1=[line.strip() for line in f]
with codecs.open('resource/polarity_words_2.txt',encoding='utf-8') as f:
    polar_words2=[line.strip() for line in f]
polar_words1.extend(polar_words2)
polar_word=set(polar_words1)
def polarword(word):
    if word in polar_word:
        return True
    else:
        return False

In [51]:
import json
with open('resource/word2cluster.json') as f:
    word2cluster=json.load(f)

In [52]:
def word_cluster(word):
    try:
        cluster=word2cluster[word]
    except KeyError:
        cluster='0000'
    return cluster

In [53]:
with codecs.open('resource/baike-50.vec.txt',encoding='utf-8') as f:
    word2emb={}
    for line in f:
        items=line.strip().split()
        emb=[float(it) for it in items[1:]]
        try:
            word2emb[items[0]]=emb
        except:
            pass

In [54]:
def add_embed(word):
    embed={}
    try:
        for ind,it in enumerate(word2emb[word]):
            embed[str(ind)]=it
    except:
        pass
    return embed

In [161]:
def word2feature(sent,i):
    sent_len=len(sent)
    word=sent[i][0]
    postag=sent[i][1]
    features={
        'word':word,
        'pref:word':word[0],
        'suff:word':word[-1],
        'postag': postag,
        'perception':perception(word),
        'polar_word':polarword(word),
        'cluster':word_cluster(word),

        # negative effect    
#         'word_head':sent[i][2],
#         'word_relation':sent[i][3],
    }
    if i>0 and i<sent_len-1:
        # about word
        word1=sent[i-1][0]
        word2=sent[i+1][0]
        features.update({
            '-1:word':word1,
            '+1:word':word2,
            '-1+word':word1+'|'+word,
            '+1+word':word+'|'+word2,
        })
        # about tag
        postag1=sent[i-1][1]
        postag2=sent[i+1][1]
        features.update({
            '-1:postag': postag1,
            '+1:postag': postag2,
            '-1+postag':postag1+'|'+postag,
            '+1+posttag':postag+'|'+postag2,
        })
    elif i==0 and len(sent)>1:
        # about word
        word2=sent[i+1][0]
        features.update({
            '+1:word':word2,
            '+1+word':word+'|'+word2,
        })
        # about tag
        postag2=sent[i+1][1]
        features.update({
            '+1:postag': postag2,
            '+1+posttag':postag+'|'+postag2,
        })
        features['EOS']=True
    elif len(sent)>1:
        # about word
        word1=sent[i-1][0]
        features.update({
            '-1:word':word1,
            '-1+word':word1+'|'+word,
        })
        # about tag
        postag1=sent[i-1][1]
        features.update({
            '-1:postag': postag1,
            '-1+postag':postag1+'|'+postag,
        })
        features['END']=True
    else:
        features['EOS']=True
        features['END']=True
    features.update(add_embed(word))
    return features
def sent2features(sent):
    return [word2feature(sent,i) for  i in range(len(sent))]
def sent2labels(sent):
    return [it[2] for it in sent]
# def sent2token(sent):
#     return [it[0] for it in sent]

In [288]:
percent=0.8
data_num=taiyi_train.shape[0]
train_num=int(data_num*percent)
random.shuffle(labels)
train_sents=labels[:train_num]
test_sents=labels[train_num:]
x_train=[]
y_train=[]
for sub_sent in train_sents:
    for sent in sub_sent:
        x_train.append(sent2features(sent))
        y_train.append(sent2labels(sent))
# x_train=[sent2features(s) for s in train_sents]
# y_train=[sent2labels(s) for s in train_sents]
x_test=[]
y_test=[]
for sub_sent in test_sents:
    for sent in sub_sent:
        x_test.append(sent2features(sent))
        y_test.append(sent2labels(sent))    
# x_test=[sent2features(s) for s in test_sents]
# y_test=[sent2labels(s) for s in test_sents]

In [289]:
%%time
crf=sklearn_crfsuite.CRF(
    algorithm='lbfgs',
#     c1=0.20,
#     c2=0.14,
    c1=1,
    c2=2,
    max_iterations=100,
    all_possible_transitions=True
                        )
crf.fit(x_train,y_train)

Wall time: 41.2 s


In [290]:
label_list=list(crf.classes_)
y_pred=crf.predict(x_test)
print(metrics.flat_classification_report(
    y_test, y_pred, label_list, digits=3))

             precision    recall  f1-score   support

          N      0.885     0.764     0.820     18678
          Y      0.926     0.968     0.946     57210

avg / total      0.916     0.917     0.915     75888



In [271]:
# import joblib
# with open('model/CRF_all.mdl','wb') as f:
#     joblib.dump(crf,f)

In [114]:
# with open('model/CRF.mdl','rb') as f:
#     crf=joblib.load(f)

# print(labels[0])
# crf.predict_single(x_train[0])

[('按键', 'n', 3, 'ATT', 'N'), ('反应', 'v', 3, 'ATT', 'N'), ('速度', 'n', 4, 'SBV', 'N'), ('有', 'v', 0, 'HED', 'N'), ('一点', 'm', 6, 'ADV', 'N'), ('慢', 'a', 4, 'VOB', 'N')]


['N', 'N', 'N', 'N', 'N', 'N']

In [77]:
# # fine tuning param
# # %%time
# crf=sklearn_crfsuite.CRF(algorithm='lbfgs',
#                         max_iterations=100,
#                         all_possible_transitions=True)
# params_space={
#     'c1':np.linspace(0,10,10),
#     'c2':np.linspace(0,10,10),
# }
# f1_score=make_scorer(metrics.flat_f1_score,average='weighted',labels=label_list)
# rs=GridSearchCV(crf,
#                      params_space,
#                      cv=5,
#                      verbose=1,
#                      n_jobs=-1,
# #                      n_iter=100,
# #                      scoring=f1_score
#                )
# rs.fit(x_train,y_train)
# print('best params:', rs.best_params_)
# print('best CV score:', rs.best_score_)
# print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))
# crf = rs.best_estimator_
# y_pred = crf.predict(x_test)
# print(metrics.flat_classification_report(
#     y_test, y_pred, label_list, digits=3
# ))

In [81]:
# sent_list=re.split('，|。|？|！',sent)
def sent_split(sent):
    sent_list=re.split('，|。|？|！',sent)
    sent_list=[it  for it in sent_list if len(it)>1]
    return sent_list

In [82]:
sent_list=sent_split(taiyi_train.loc[24][1])
sent_list

['清洁功能挺好', '以前就算刚洗完头发不久就有粉粉的头屑', '用这个洗就一点都没有了']

In [223]:
def sent_compress(row):
#     sent_label=[]
    sent_comp=[]
    sent_feature=sent2features(row)
    label=crf.predict_single(sent_feature)
#     sent_label.append(label)
    for ind,lb in enumerate(label):
        if lb == 'N':
            sent_comp.append(row[ind][0])
    return (sent_comp,label)

In [291]:
#train set 
sent_press_list=[]
sent_label_list=[]
for ind,row in enumerate(train['orig_seg']):
    try:
        sent_press=[]
        sent_label=[]
        for sub_sent in row:
            result=sent_compress(sub_sent)
            sent_press.append(result[0])
            sent_label.append(result[1])
        sent_press_list.append(sent_press)
        sent_label_list.append(sent_label)
    except:
        pass

train['sent_compress']=sent_press_list
train['sent_label']=sent_label_list

# taiyi_train.drop('row_id',axis=1,inplace=True)
# taiyi_train.to_csv('taiyi_train_compress.csv',index=None,encoding='utf-8')

In [297]:
#test set 
sent_press_list=[]
sent_label_list=[]
for ind,row in enumerate(test['orig_seg']):
    try:
        sent_press=[]
        sent_label=[]
        for sub_sent in row:
            result=sent_compress(sub_sent)
            sent_press.append(result[0])
            sent_label.append(result[1])
        sent_press_list.append(sent_press)
        sent_label_list.append(sent_label)
    except:
        pass

test['sent_compress']=sent_press_list
test['sent_label']=sent_label_list

# taiyi_train.drop('row_id',axis=1,inplace=True)
# taiyi_train.to_csv('taiyi_train_compress.csv',index=None,encoding='utf-8')

In [298]:
# sent_press_list[0],sent_label_l
with open('sent_label_test.json','w') as f:
    json.dump(sent_label_list,f)

In [255]:
test[:2]

,content,tokens,postags,orig_seg,sent_compress,sent_label
0,比黑人碳头牙刷好用?,"[[比, 黑人, 碳头, 牙刷, 好, 用]]","[[p, n, n, n, a, v]]","[[(比, p), (黑人, n), (碳头, n), (牙刷, n), (好, a), (...","[[牙刷, 好]]","[[Y, Y, Y, N, N, Y]]"
1,比之前坏的要好些，亮一些。,"[[比, 之前, 坏, 的, 要, 好, 些], [亮, 一些]]","[[p, nd, a, u, v, a, q], [v, m]]","[[(比, p), (之前, nd), (坏, a), (的, u), (要, v), (好...","[[坏, 好], []]","[[Y, Y, N, Y, Y, N, Y], [Y, Y]]"


In [219]:
train.to_excel('taiyi_train_compress.xls',index=None,encoding='utf-8')

In [81]:
#test set
sent_press=[]
sent_no_press=[]
sent_label=[]
for ind,row in enumerate(taiyi_test[taiyi_test.columns[1]]):
    try:
        result=sent_compress(row)
        sent_press.append(result[0])
        sent_no_press.append(result[1])
        sent_label.append(result[2])
    except:
        sent_press.append([])
        sent_no_press.append([])
        sent_label.append([])
taiyi_test['sent_compress']=sent_press
taiyi_test['sent_no_press']=sent_no_press
taiyi_test['sent_label']=sent_label

In [85]:
# taiyi_test.drop('row_id',axis=1,inplace=True)
taiyi_test.to_excel('taiyi_test_compress.xls',index=None,encoding='utf-8')